# 10월29일

# 주가데이터

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
### naver증권 - 텍스트(일부) 입력 -> 텍스트를 포함하는 테마명 반환
def theme_list(text):
    theme = []
    url = f'https://finance.naver.com/sise/theme.naver'
    with requests.get(url) as doc:
        html = BeautifulSoup(doc.text, "lxml")
        pgrr = html.find('td', class_='pgRR')
        last_page = str(pgrr.a['href']).split('=')[1]

    n=1
    while n<=int(last_page):
        url = f'https://finance.naver.com/sise/theme.naver?page={n}'
        with requests.get(url) as doc:
            html = BeautifulSoup(doc.text, "lxml")
            pgrr = html.find_all('td', class_='col_type1')
            for i in pgrr:
                theme.append(i.text)
        n+=1
    theme_df = pd.DataFrame(theme,columns=['테마'])
    
    if len(theme_df[theme_df['테마'].str.contains(text)]) <= 0:
        print('입력한 텍스트를 포함한 테마명이 없습니다.')
    else:
        return theme_df[theme_df['테마'].str.contains(text)]

In [ ]:
### naver증권 - 테마 입력 시 회사명 리스트 반환(수정 후)
def corp_in_theme(theme):
    driver = webdriver.Chrome('/Users/hyunjeongjo/desktop/chrome/chromedriver')
    driver.get(f'https://finance.naver.com/sise/theme.naver')
    pgrr = driver.find_element_by_link_text('맨뒤').get_attribute('href')
    last_page = str(pgrr).split('=')[1]
    for i in range(1,int(last_page)):
        driver.get(f'https://finance.naver.com/sise/theme.naver?&page={i}')
        try:
            element = driver.find_element_by_link_text(theme)
            element.send_keys(Keys.RETURN)
            corpnames = driver.find_elements_by_class_name('name')
            corp = []
            for name in corpnames:
                corp.append(name.text.strip('* '))
            driver.close()
            return corp
        except:
            continue    

In [ ]:
# 회사명, 종목코드, 업종, 주요제품 등 반환
def corp_df():
    code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]
    code_df['종목코드'] = code_df['종목코드'].map('{:06d}'.format)
    return code_df

# 회사명 일부 입력 -> 입력한 회사명 일부가 포함된 상장된 회사 정보를 반환
def corp_info(corp):
    code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]
    code_df['종목코드'] = code_df['종목코드'].map('{:06d}'.format)
    return code_df[code_df['회사명'].str.contains(corp)]

# 회사명 입력 시 종목코드 반환
def search_corp_code(corp):
    code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]
    code_df['종목코드'] = code_df['종목코드'].map('{:06d}'.format)
    code = code_df.query(f'회사명=="{corp}"')['종목코드'].values
    return code

# 회사명 입력 시 주가데이터 데이터프레임 반환

def stock_data(corp):
    code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]
    code_df['종목코드'] = code_df['종목코드'].map('{:06d}'.format)
    code = code_df.query(f'회사명=="{corp}"')['종목코드'].values[0]

    url = f'https://finance.naver.com/item/sise_day.nhn?code={code}&page=1'
    with requests.get(url) as doc:
        html = BeautifulSoup(doc.text, "lxml")
        pgrr = html.find('td', class_='pgRR')
        s = str(pgrr.a['href']).split('=')
        last_page = s[-1]

    df = pd.DataFrame()
    sise_url = f'https://finance.naver.com/item/sise_day.nhn?code={code}'

    for page in range(1, int(last_page)+1):
        page_url = '{}&page={}'.format(sise_url, page)
        response_page = requests.get(page_url).text
        df = df.append(pd.read_html(response_page)[0])

    df = df.dropna()
    df = df.reset_index(drop=True)
    return df

# 주가 데이터 엑셀파일로 저장
# 여러 회사 입력 시 시트에 회사명으로 저장

def stock_data_to_excel(*corp):
    writer = pd.ExcelWriter('stock_data.xlsx',engine='xlsxwriter') # 엑셀
    code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]
    code_df['종목코드'] = code_df['종목코드'].map('{:06d}'.format)
    
    for n in range(len(corp)):
        code = code_df.query(f'회사명=="{corp[n]}"')['종목코드'].values[0]
        url = f'https://finance.naver.com/item/sise_day.nhn?code={code}&page=1'
        with requests.get(url) as doc:
            html = BeautifulSoup(doc.text, "lxml")
            pgrr = html.find('td', class_='pgRR')
            s = str(pgrr.a['href']).split('=')
            last_page = s[-1]
            
        df = pd.DataFrame()
        sise_url = f'https://finance.naver.com/item/sise_day.nhn?code={code}'
        for page in range(1, int(last_page)+1):
            page_url = '{}&page={}'.format(sise_url, page)
            response_page = requests.get(page_url, headers={'User-agent': 'Mozilla/5.0'}).text
            df = df.append(pd.read_html(response_page)[0])
            
        df = df.dropna()
        df = df.reset_index(drop=True)
        df.to_excel(writer, sheet_name=corp[n])
        display(f'<{corp[n]}>',df)
        
    writer.save()
    print('엑셀파일 저장 완료')    

### headers User-agent 추가 생략함